In [1]:
import pdb
from math import log10, floor

import numpy as np
import scipy.optimize as op
import scipy.special as spec
from scipy import stats
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import seaborn as sns
import corner

import astropy
from astropy import stats as astrostats

import emcee
import radvel

%matplotlib inline

In [30]:
sys_props   = pd.read_csv('legacy_tables/system_props_719.csv')
stellar     = pd.read_csv('/Users/lee/Academics/Astronomy/Planets/legacy_analysis/legacy_tables/legacy_specmatch_medians.csv')
masses_only = pd.read_csv('legacy_tables/planet_list_719.csv')
trends      = pd.read_csv('legacy_tables/trend_list_719.csv')

### Replace NaNs with dummy values, for now.

In [3]:
stellar = stellar.fillna(100.5)

In [4]:
stellar.iso_radius

0      10.872633
1       0.995520
2       0.847684
3       1.134055
4       0.705415
         ...    
740     1.125570
741     1.339110
742     0.521268
743     0.644921
744     1.885604
Name: iso_radius, Length: 745, dtype: float64

# Make table for planet parameters.

In [57]:
masses_real = masses_only.query('status != "N" and status != "A"').reset_index(drop=True)
masses_good = masses_real.query('mass != "nan"').reset_index(drop=True)#[masses_only.hostname != 'hip63510']
masses_binary = masses_good.query('status == "S" or status == "SS"').reset_index(drop=True)
masses_planet = masses_good.query('status != "S" and status != "SS"').reset_index(drop=True)
masses_old  = masses_planet.query('status == "K"').reset_index(drop=True)
masses_new  = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)

In [58]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_planet['hostname']
masses_table['Msini (MJup)'] = masses_planet['mass_med']
masses_table['a (AU)'] = masses_planet['axis_med']
masses_table['P (d)'] = masses_planet['per_med']
masses_table['e'] = masses_planet['e_med']

names = np.unique(np.array(masses_table.Name))

nplanets = np.array(masses_table.groupby('Name').count()['e'])
db_nplanets = pd.DataFrame({'Name':names, 'nplanets':nplanets})
masses_table = pd.merge(masses_table, db_nplanets, on='Name').reset_index()

In [59]:
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_planet.hostname))
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1
        
masses_planet['Name'] = masses_table['Name']

## Do the same for stellar binaries.

In [60]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_binary['hostname']
masses_table['Msini (MJup)'] = masses_binary['mass_med']
masses_table['a (AU)'] = masses_binary['axis_med']
masses_table['P (d)'] = masses_binary['per_med']
masses_table['e'] = masses_binary['e_med']

names = np.unique(np.array(masses_table.Name))

nbinarys = np.array(masses_table.groupby('Name').count()['e'])
db_nbinarys = pd.DataFrame({'Name':names, 'nbinarys':nbinarys})
masses_table = pd.merge(masses_table, db_nbinarys, on='Name').reset_index()

In [61]:
'''
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_binary.hostname))
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1
'''       
masses_binary['Name'] = masses_table['Name'].str.upper()

In [5]:
def round_to_1(x):
    if np.isnan(x):
        return 0.0001
    else:
        return round(x, -int(floor(log10(x))))

def round_sig(x, sig=2):
    if x == np.nan:
        return np.nan
    else:
        return round(x, sig-int(floor(log10(abs(x)))) - 1) 
round_sig_vec = np.vectorize(round_sig)

def round_to_reference(x, y):
    if x == np.nan or y == np.nan:
        return np.nan
    else:
        return round(x, -int(floor(log10(abs(y)))) + 1)
round_to_ref_vec = np.vectorize(round_to_reference)

## Make table for planet parameters.

In [63]:
# Round errors and parameters with significant figures.
masses_planet['mass_err_plus_r']  = round_sig_vec(np.array(masses_planet['mass_plus'] - masses_planet['mass_med']))
masses_planet['mass_err_minus_r'] = round_sig_vec(np.array(masses_planet['mass_med'] - masses_planet['mass_minus']))
masses_planet['mass_err_av']      = round_sig_vec(0.5*(masses_planet['mass_err_plus_r'] + masses_planet['mass_err_minus_r']))
masses_planet['mass_med_r']       = round_to_ref_vec(np.array(masses_planet['mass_med']), masses_planet['mass_err_av'])

masses_planet['axis_err_plus_r']  = round_sig_vec(np.array(masses_planet['axis_plus'] - masses_planet['axis_med']))
masses_planet['axis_err_minus_r'] = round_sig_vec(np.array(masses_planet['axis_med'] - masses_planet['axis_minus']))
masses_planet['axis_err_av']      = round_sig_vec(0.5*(masses_planet['axis_err_plus_r'] + masses_planet['axis_err_minus_r']))
masses_planet['axis_med_r']       = round_to_ref_vec(np.array(masses_planet['axis_med']), masses_planet['axis_err_av'])

masses_planet['per_err_plus_r']   = round_sig_vec(np.array(masses_planet['per_plus'] - masses_planet['per_med']))
masses_planet['per_err_minus_r']  = round_sig_vec(np.array(masses_planet['per_med'] - masses_planet['per_minus']))
masses_planet['per_err_av']       = round_sig_vec(0.5*(masses_planet['per_err_plus_r'] + masses_planet['per_err_minus_r']))
masses_planet['per_med_r']        = round_to_ref_vec(np.array(masses_planet['per_med']), masses_planet['per_err_av'])

masses_planet['e_err_plus_r']     = round_sig_vec(np.array(masses_planet['e_plus'] - masses_planet['e_med']))
masses_planet['e_err_minus_r']    = round_sig_vec(np.array(masses_planet['e_med'] - masses_planet['e_minus']))
masses_planet['e_err_av']         = round_sig_vec(0.5*(masses_planet['e_err_plus_r'] + masses_planet['e_err_minus_r']))
masses_planet['e_med_r']          = round_to_ref_vec(np.array(masses_planet['e_med']), masses_planet['e_err_av'])
'''
masses_planet['tc_err_plus_r']     = round_sig_vec(np.array(masses_planet['tc_plus'] - masses_planet['tc_med']))
masses_planet['tc_err_minus_r']    = round_sig_vec(np.array(masses_planet['tc_med'] - masses_planet['tc_minus']))
masses_planet['tc_err_av']         = round_sig_vec(0.5*(masses_planet['tc_err_plus_r'] + masses_planet['tc_err_minus_r']))
masses_planet['tc_med_r']          = round_to_ref_vec(np.array(masses_planet['tc_med']), masses_planet['tc_err_av'])

masses_planet['w_err_plus_r']     = round_sig_vec(np.array(masses_planet['w_plus'] - masses_planet['w_med']))
masses_planet['w_err_minus_r']    = round_sig_vec(np.array(masses_planet['w_med'] - masses_planet['w_minus']))
masses_planet['w_err_av']         = round_sig_vec(0.5*(masses_planet['w_err_plus_r'] + masses_planet['w_err_minus_r']))
masses_planet['w_med_r']          = round_to_ref_vec(np.array(masses_planet['w_med']), masses_planet['w_err_av'])
'''
tex_writer = open('legacy_tables/planet_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in masses_planet.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['mass_med_r'], 
                                                   row['mass_err_plus_r'], 
                                                   row['mass_err_minus_r']) + r'$'
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['axis_med_r'], 
                                                   row['axis_err_plus_r'], 
                                                   row['axis_err_minus_r']) + r'$'
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['per_med_r'], 
                                                   row['per_err_plus_r'], 
                                                   row['per_err_minus_r']) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['e_med_r'], 
                                                   row['e_err_plus_r'], 
                                                   row['e_err_minus_r']) + r'$'
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

# Make table for stellar parameters. TO-DO: COMBINE SYNTHETIC/EMPIRICAL SPECTRAL PARAMETERS, USE APPROPRIATE SET FOR EACH STAR. 

## THIS COMBINATION SHOULD BE DONE IN THE SPECMATCH SELECTION NOTEBOOK.

In [64]:
stellar.teff_err_emp

0      110.0
1      110.0
2      110.0
3      110.0
4      110.0
       ...  
740    110.0
741    110.0
742     70.0
743     70.0
744    110.0
Name: teff_err_emp, Length: 745, dtype: float64

In [65]:
# Specmatch parameters first. TO-DO: COMBINE SYNTHETIC/EMPIRICIAL RESULTS. MAKE COLUMN WITH SYN/EMP
stellar['teff_err_r'] = round_sig_vec(stellar['teff_err_emp'])
stellar['teff_r'] = round_to_ref_vec(np.array(stellar['teff_emp']), stellar['teff_err_r'])
stellar['fe_err_r'] = round_sig_vec(stellar['fe_err'])
stellar['fe_r'] = round_to_ref_vec(np.array(stellar['fe']), stellar['fe_err_r'])

# USE ISOCLASSIFY OUTPUTS FOR R AND LOGG, SINCE SPECMATCH-SYN AND -EMP EACH ONLY PRODUCE ONE. 
# FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX
stellar['logg_err_r'] = round_sig_vec(stellar['logg_err'])
stellar['logg_r'] = round_to_ref_vec(np.array(stellar['logg']), stellar['logg_err_r'])
stellar['radius_err_r'] = round_sig_vec(0.5*(stellar['iso_radius_err1'] + stellar['iso_radius_err2']))
stellar['radius_r'] = round_to_ref_vec(np.array(stellar['iso_radius']), stellar['radius_err_r'])

# Now isoclassify.
stellar['mass_err_r'] = round_sig_vec(0.5*(stellar['iso_mass_err1'] + stellar['iso_mass_err2']))
stellar['mass_r'] = round_to_ref_vec(np.array(stellar['iso_mass']), stellar['mass_err_r'])


tex_writer = open('legacy_tables/stellar_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & $T_{\textrm{eff}}$ [K] & [Fe/H] & log$g$ [log(cm s$^{-2}$)] & $M$ [\msun\] & $R$ [\rsun\] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in stellar.iterrows():
    # Customize names.
    namein = str(row['name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    teff_str   = r'$' + '{0}\\pm {1}'.format(row['teff_r'], row['teff_err_r']) + r'$'    
    fe_str     = r'$' + '{0}\\pm {1}'.format(row['fe_r'], row['fe_err_r']) + r'$'    
    logg_str   = r'$' + '{0}\\pm {1}'.format(row['logg_r'], row['logg_err_r']) + r'$' 
    radius_str = r'$' + '{0}\\pm {1}'.format(row['radius_r'], row['radius_err_r']) + r'$'      
    mass_str   = r'$' + '{0}\\pm {1}'.format(row['mass_r'], row['mass_err_r']) + r'$'

    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, teff_str, fe_str, logg_str, radius_str, mass_str))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make new candidate table. Preliminary as of 11/23/19.

In [66]:
masses_candidate = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)

In [67]:
# Round errors and parameters with significant figures.
masses_candidate['mass_err_plus_r']  = round_sig_vec(np.array(masses_candidate['mass_plus'] - masses_candidate['mass_med']))
masses_candidate['mass_err_minus_r'] = round_sig_vec(np.array(masses_candidate['mass_med'] - masses_candidate['mass_minus']))
masses_candidate['mass_err_av']      = round_sig_vec(0.5*(masses_candidate['mass_err_plus_r'] + masses_candidate['mass_err_minus_r']))
masses_candidate['mass_med_r']       = round_to_ref_vec(np.array(masses_candidate['mass_med']), masses_candidate['mass_err_av'])

masses_candidate['axis_err_plus_r']  = round_sig_vec(np.array(masses_candidate['axis_plus'] - masses_candidate['axis_med']))
masses_candidate['axis_err_minus_r'] = round_sig_vec(np.array(masses_candidate['axis_med'] - masses_candidate['axis_minus']))
masses_candidate['axis_err_av']      = round_sig_vec(0.5*(masses_candidate['axis_err_plus_r'] + masses_candidate['axis_err_minus_r']))
masses_candidate['axis_med_r']       = round_to_ref_vec(np.array(masses_candidate['axis_med']), masses_candidate['axis_err_av'])

masses_candidate['per_err_plus_r']   = round_sig_vec(np.array(masses_candidate['per_plus'] - masses_candidate['per_med']))
masses_candidate['per_err_minus_r']  = round_sig_vec(np.array(masses_candidate['per_med'] - masses_candidate['per_minus']))
masses_candidate['per_err_av']       = round_sig_vec(0.5*(masses_candidate['per_err_plus_r'] + masses_candidate['per_err_minus_r']))
masses_candidate['per_med_r']        = round_to_ref_vec(np.array(masses_candidate['per_med']), masses_candidate['per_err_av'])

masses_candidate['e_err_plus_r']     = round_sig_vec(np.array(masses_candidate['e_plus'] - masses_candidate['e_med']))
masses_candidate['e_err_minus_r']    = round_sig_vec(np.array(masses_candidate['e_med'] - masses_candidate['e_minus']))
masses_candidate['e_err_av']         = round_sig_vec(0.5*(masses_candidate['e_err_plus_r'] + masses_candidate['e_err_minus_r']))
masses_candidate['e_med_r']          = round_to_ref_vec(np.array(masses_candidate['e_med']), masses_candidate['e_err_av'])
'''
masses_candidate['tc_err_plus_r']     = round_sig_vec(np.array(masses_candidate['tc_plus'] - masses_candidate['tc_med']))
masses_candidate['tc_err_minus_r']    = round_sig_vec(np.array(masses_candidate['tc_med'] - masses_candidate['tc_minus']))
masses_candidate['tc_err_av']         = round_sig_vec(0.5*(masses_candidate['tc_err_plus_r'] + masses_candidate['tc_err_minus_r']))
masses_candidate['tc_med_r']          = round_to_ref_vec(np.array(masses_candidate['tc_med']), masses_candidate['tc_err_av'])

masses_candidate['w_err_plus_r']     = round_sig_vec(np.array(masses_candidate['w_plus'] - masses_candidate['w_med']))
masses_candidate['w_err_minus_r']    = round_sig_vec(np.array(masses_candidate['w_med'] - masses_candidate['w_minus']))
masses_candidate['w_err_av']         = round_sig_vec(0.5*(masses_candidate['w_err_plus_r'] + masses_candidate['w_err_minus_r']))
masses_candidate['w_med_r']          = round_to_ref_vec(np.array(masses_candidate['w_med']), masses_candidate['w_err_av'])
'''
tex_writer = open('legacy_tables/candidate_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in masses_candidate.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['mass_med_r'], 
                                                   row['mass_err_plus_r'], 
                                                   row['mass_err_minus_r']) + r'$'
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['axis_med_r'], 
                                                   row['axis_err_plus_r'], 
                                                   row['axis_err_minus_r']) + r'$'
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['per_med_r'], 
                                                   row['per_err_plus_r'], 
                                                   row['per_err_minus_r']) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['e_med_r'], 
                                                   row['e_err_plus_r'], 
                                                   row['e_err_minus_r']) + r'$'
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make stellar/substellar binary table.

In [68]:
masses_binary = masses_binary.fillna(10.5)
masses_binary = masses_binary.query('mass_med > 0 and mass_minus > 0 and mass > 0 and \
                                     k_med > 0 and k_minus > 0 and k > 0 and \
                                     axis_med > 0 and axis_minus > 0 and axis > 0 and \
                                     per_med > 0 and per_minus > 0 and per > 0').reset_index()

In [69]:
masses_binary

,index,Unnamed: 0,hostname,status,mass,mass_med,mass_minus,mass_plus,axis,axis_med,...,per_plus,k,k_med,k_minus,k_plus,e,e_med,e_minus,e_plus,Name
0,0,19,10790,S,276.570990,283.440200,263.338796,310.707878,1.541407,1.541341,...,6.943887e+02,10039.751087,10357.537875,9481.818162,11664.070025,0.843585,0.847259,0.836583,0.860889,10790
1,1,23,112914,S,221.273367,221.291795,216.032621,226.400052,1.382148,1.382217,...,7.095718e+02,5671.991830,5671.923251,5669.981287,5673.912290,0.324168,0.324166,0.323802,0.324526,112914
2,3,32,116321,SS,50.662192,130.550149,74.569845,226.267222,7.726527,16.282146,...,5.909878e+04,435.279112,844.464984,566.243664,1810.602398,0.107801,0.641658,0.308283,0.825405,116321
3,4,36,120136,S,138.260897,124.055815,76.971755,221.421705,22.329765,18.229948,...,6.489907e+04,1495.816708,1460.914565,753.852810,2587.178152,0.882998,0.883455,0.769510,0.955908,120136
4,5,57,142229,S,234.820839,707.506840,339.424375,1314.522195,13.628901,24.469100,...,8.029630e+04,1768.473156,3636.512863,2274.056395,5356.987033,0.483839,0.637092,0.522294,0.736410,142229
5,6,58,142267,S,140.124950,140.132549,136.460587,143.744275,0.490494,0.490507,...,1.385797e+02,6788.759537,6788.696980,6786.425523,6791.028344,0.547547,0.547528,0.547262,0.547803,142267
6,8,71,157338,SS,124.416924,146.258736,99.582039,199.964474,17.543707,24.328158,...,7.103505e+04,990.658489,1064.700778,752.437988,1436.283555,0.605454,0.680955,0.610324,0.748611,157338
7,9,73,159062,S,29.909743,259.190030,44.537921,825.600001,17.969802,57.096979,...,3.414346e+05,238.285271,1438.746531,619.217748,3683.647371,0.265009,0.317443,0.102451,0.642134,159062
8,10,76,16160,S,65.356742,66.020408,64.284999,67.733944,15.790276,16.307961,...,2.886365e+04,711.241160,713.103810,712.087797,714.132301,0.636619,0.644657,0.640455,0.648939,16160
9,11,77,161797,S,63.760984,104.860296,81.968194,130.841016,15.053919,16.180753,...,2.417266e+04,450.468348,679.932150,554.866526,813.806653,0.303640,0.314776,0.299387,0.334787,161797


In [70]:
masses_binary['mass_err_plus_r']  = round_sig_vec(np.abs(np.array(masses_binary['mass_plus'] - masses_binary['mass_med'])))
masses_binary['mass_err_minus_r'] = round_sig_vec(np.abs(np.array(masses_binary['mass_med'] - masses_binary['mass_minus'])))
masses_binary['mass_err_av']      = round_sig_vec(0.5*(masses_binary['mass_err_plus_r'] + masses_binary['mass_err_minus_r']))
masses_binary['mass_med_r']       = round_to_ref_vec(np.array(masses_binary['mass_med']), masses_binary['mass_err_av'])

masses_binary['axis_err_plus_r']  = round_sig_vec(np.abs(np.array(masses_binary['axis_plus'] - masses_binary['axis_med'])))
masses_binary['axis_err_minus_r'] = round_sig_vec(np.abs(np.array(masses_binary['axis_med'] - masses_binary['axis_minus'])))
masses_binary['axis_err_av']      = round_sig_vec(0.5*(masses_binary['axis_err_plus_r'] + masses_binary['axis_err_minus_r']))
masses_binary['axis_med_r']       = round_to_ref_vec(np.array(masses_binary['axis_med']), masses_binary['axis_err_av'])

masses_binary['per_err_plus_r']   = round_sig_vec(np.abs(np.array(masses_binary['per_plus'] - masses_binary['per_med'])))
masses_binary['per_err_minus_r']  = round_sig_vec(np.abs(np.array(masses_binary['per_med'] - masses_binary['per_minus'])))
masses_binary['per_err_av']       = round_sig_vec(0.5*(masses_binary['per_err_plus_r'] + masses_binary['per_err_minus_r']))
masses_binary['per_med_r']        = round_to_ref_vec(np.array(masses_binary['per_med']), masses_binary['per_err_av'])

masses_binary['e_err_plus_r']     = round_sig_vec(np.abs(np.array(masses_binary['e_plus'] - masses_binary['e_med'])))
masses_binary['e_err_minus_r']    = round_sig_vec(np.abs(np.array(masses_binary['e_med'] - masses_binary['e_minus'])))
masses_binary['e_err_av']         = round_sig_vec(0.5*(masses_binary['e_err_plus_r'] + masses_binary['e_err_minus_r']))
masses_binary['e_med_r']          = round_to_ref_vec(np.array(masses_binary['e_med']), masses_binary['e_err_av'])
'''
masses_binary['tc_err_plus_r']     = round_sig_vec(np.array(masses_binary['tc_plus'] - masses_binary['tc_med']))
masses_binary['tc_err_minus_r']    = round_sig_vec(np.array(masses_binary['tc_med'] - masses_binary['tc_minus']))
masses_binary['tc_err_av']         = round_sig_vec(0.5*(masses_binary['tc_err_plus_r'] + masses_binary['tc_err_minus_r']))
masses_binary['tc_med_r']          = round_to_ref_vec(np.array(masses_binary['tc_med']), masses_binary['tc_err_av'])

masses_binary['w_err_plus_r']     = round_sig_vec(np.array(masses_binary['w_plus'] - masses_binary['w_med']))
masses_binary['w_err_minus_r']    = round_sig_vec(np.array(masses_binary['w_med'] - masses_binary['w_minus']))
masses_binary['w_err_av']         = round_sig_vec(0.5*(masses_binary['w_err_plus_r'] + masses_binary['w_err_minus_r']))
masses_binary['w_med_r']          = round_to_ref_vec(np.array(masses_binary['w_med']), masses_binary['w_err_av'])
'''
tex_writer = open('legacy_tables/binary_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in masses_binary.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h' or namein[0] == 'H':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g' or namein[0] == 'G':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b' or namein[0] == 'B':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['mass_med_r'], 
                                                   row['mass_err_plus_r'], 
                                                   row['mass_err_minus_r']) + r'$'
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['axis_med_r'], 
                                                   row['axis_err_plus_r'], 
                                                   row['axis_err_minus_r']) + r'$'
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['per_med_r'], 
                                                   row['per_err_plus_r'], 
                                                   row['per_err_minus_r']) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['e_med_r'], 
                                                   row['e_err_plus_r'], 
                                                   row['e_err_minus_r']) + r'$'
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

ValueError: math domain error

## Make table for trend and curvature terms.

In [31]:
trends_curv = trends.query('curv == curv')
#pdb.set_trace()

# Round errors and parameters with significant figures.
trends['dvdt_err_plus_r']  = round_sig_vec(np.array(trends['dvdt_plus'] - trends['dvdt_med']))
trends['dvdt_err_minus_r'] = round_sig_vec(np.array(trends['dvdt_med'] - trends['dvdt_minus']))
trends['dvdt_err_av']      = round_sig_vec(0.5*(trends['dvdt_err_plus_r'] + trends['dvdt_err_minus_r']))
trends['dvdt_med_r']       = round_to_ref_vec(np.array(trends['dvdt_med']), trends['dvdt_err_av'])

# Combine curvature and non-curvature databases.
trends_curv['curv_err_plus_r']  = round_sig_vec(np.array(trends_curv['curv_plus'] - trends_curv['curv_med']))
trends_curv['curv_err_minus_r'] = round_sig_vec(np.array(trends_curv['curv_med'] - trends_curv['curv_minus']))
trends_curv['curv_err_av']      = round_sig_vec(0.5*(trends_curv['curv_err_plus_r'] + trends_curv['curv_err_minus_r']))
trends_curv['curv_med_r']       = round_to_ref_vec(np.array(trends_curv['curv_med']), trends_curv['curv_err_av'])

#trends = trends.merge(trends_curv, on='hostname', suffixes=('', '_curv'))
#trends = trends.update(trends_curv)
#pdb.set_trace()

tex_writer = open('legacy_tables/trends_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & dvdt & curvature \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in trends.iterrows():
    # Customize names.
    basename = str(row['hostname'])
    namein = str(row['hostname'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    dvdtin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['dvdt_med_r'], 
                                                   row['dvdt_err_plus_r'], 
                                                   row['dvdt_err_minus_r']) + r'$'
    if basename in trends_curv['hostname'].tolist():
        row = trends_curv.query('hostname == "{}"'.format(basename))
        curvin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['curv_med_r'].to_list()[0], 
                                                       row['curv_err_plus_r'].to_list()[0], 
                                                       row['curv_err_minus_r'].to_list()[0]) + r'$'
        #pdb.set_trace()
    else:
        curvin = '0'

    
    tex_writer.write('{0} & {1} & {2} \\\\ \n'.format(namein, dvdtin, curvin))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()